1. 네이버 증권의 업종 분류를 가져와 적용시키는 곳

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime

In [5]:
# 오늘 날짜 가져오기
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')
new_file_name = f'sector_info_df_{today}.csv'

In [9]:
sector_df = pd.read_csv(f'../../crawling/{new_file_name}')


In [11]:
sector_df.head(1)

,업종코드,업종명,전일비,전체,상승,보합,하락,등락그래프
0,281,건강관리장비와용품,+1.17%,95,27,11,57,9%


In [15]:
def sector_detail_url(sector_name):
  '''
  업종명으로 해당업종의 상세목록을 볼수있는 url을 반환하는 함수
  :param sector_name: 업종명
  :return: 해당업종명에 해당하는 종목리스트를 볼수있는 주소
  '''
  sector_code = sector_df[sector_df['업종명'] == sector_name]['업종코드'].values[0]
  if sector_code.size == 0:
    print('존재하지 않는 업종입니다.')
    return None
  # print(type(sector_code))      #numpy.int64
  url = f'https://finance.naver.com/sise/sise_group_detail.naver?type=upjong&no={sector_code}'
  return url

In [ ]:
sector_list = sector_df['업종명'].tolist()
sector_list

In [ ]:
# 파일의 업종명을 돌면서, 해당 업종내 종목, 코스피/코스닥 유무 분류.
# 업종명에 접속->

sector_by_stock_list = []

for sector in sector_list :
   url = sector_detail_url(sector)
   res = requests.get(url)
   if res.status_code != 200:
     print('데이터를 가져오는데 실패했습니다.')
     exit()
  #  print(res.status_code)
   
   soup = BeautifulSoup(res.content, 'lxml')
   rows = soup.select('#contentarea > div:nth-child(5) > table > tbody > tr')
   
   for tr in rows :
     
    cols = tr.find_all("td")                        # BeautifulSoup 환경
    # cols = tr.find_elements(By.TAG_NAME, "td")    # Selenium 환경
    if len(cols) >= 9:
      stock_name = cols[0].text.strip()
      is_kosdaq = '*' in stock_name  # 별표가 있으면 True, 없으면 False
      stock_info = {
      '업종명': sector,  
      '종목명':stock_name.replace('*', '').strip(),  # * 기호 제거
      '코스닥 여부': is_kosdaq  # 코스닥 여부 추가
      }
      sector_by_stock_list.append(stock_info)

sector_by_stock_list_df = pd.DataFrame(sector_by_stock_list)
sector_by_stock_list_df.to_csv('sector_by_stock_list_df.csv', index=False, encoding='utf-8-sig')
